In [49]:
%pip install nltk swifter swifter[notebook]

zsh:1: no matches found: swifter[notebook]


Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import random
import string
import swifter
import torch

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
# Load csv file
df = pd.read_csv('testing-song-lyrics.csv')

In [3]:
print(f"number of songs: {len(df)}")
df.head()

number of songs: 12295


,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


# Preprocessing

In [4]:
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/ffactory/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ffactory/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
(len(df[df["language"] == "en"]), len(df))

(12064, 12295)

In [6]:
ps = PorterStemmer()
stopwords_en = set(stopwords.words('english'))

# Create a copy of the dataframe
df_proc = df.copy()

# Select only english songs
df_proc = df[df["language"] == "en"]

# Select columns we care about
df_proc = df_proc[["title","lyrics"]]
# Rename column
df_proc.rename(columns={"lyrics": "tokens"}, inplace=True)

# Convert to lowercase
df_proc["tokens"] = df_proc["tokens"].str.lower()
# Remove text between square brackets and any non alphanumeric / whitespace characters
df_proc["tokens"] = df_proc["tokens"].str.replace(re.compile(r"\[.{0,100}\]|[^\w\s]"), "", regex=True)
# Split text into words
df_proc["tokens"] = df_proc["tokens"].str.rsplit()
# Remove stopwords and stem tokens
def remove_stopwords_and_stem(tokens):
    return [ps.stem(token) for token in tokens if token not in stopwords_en]
df_proc["tokens"] = df_proc["tokens"].swifter.apply(remove_stopwords_and_stem)

Pandas Apply:   0%|          | 0/12064 [00:00<?, ?it/s]

In [ ]:
(", ".join(df_proc.iloc[5]["Tokens"][:20]), df.iloc[5]["lyrics"])

('haha, uhhuh, homo, young, mula, babi, say, he, sweet, make, wanna, lick, wrapper, remix, babi, lollipop, lollipop, breasts, like, dolli',
 '[Intro: Lil Wayne]\nHaha\nUh-huh\nNo homo (Young Mula, baby!)\nI say, he\'s so sweet, make her wanna lick the wrapper\nRemix, baby!\n\n[Verse 1: Kanye West]\nLollipop, lollipop, breastses just like Dolly Parton\nShe ride my spaceship \'til she hit the top\nThat hit the spot\n\'Til she ask, "How many li-i-li-i-licks do it take" \'til she get to shop?\nDon\'t worry why my wrists got so freeze\nTell a girl, "Like Doritos, that\'s not \'cho cheese"\nTell her friends, "Like Fritos, I\'m tryin\' to lay"\nI can\'t only have one, and I ain\'t trying to wait\nThis a song with Wayne, so you know it\'s gon\' melt\nBut you ain\'t finna murder me like everybody else\nI\'ma rap like I got some type respect for myself\nI don\'t do it for my health, man, I do it for the belt\nMan, I do it to the death, \'til the roof get melt\nHundred degrees, drop the roof, so 

In [12]:
df_proc.head()

,title,tokens
0,Killa Cam,"[killa, cam, killa, cam, cam, killa, cam, kill..."
1,Can I Live,"[yeah, hah, yeah, rocafella, invit, somethin, ..."
2,Forgive Me Father,"[mayb, caus, im, eatin, bastard, fiend, grub, ..."
3,Down and Out,"[ugh, killa, babi, kany, 1970, heron, flow, hu..."
4,Fly In,"[ask, young, boy, gon, second, time, around, g..."


In [13]:
# Save the testing-song-lyrics.csv file as a pickle file
torch.save(df_proc, 'subset_documents.pkl')

# Test if the pickle file is saved correctly
df_reloaded = torch.load('subset_documents.pkl')
df_reloaded.head()

,title,tokens
0,Killa Cam,"[killa, cam, killa, cam, cam, killa, cam, kill..."
1,Can I Live,"[yeah, hah, yeah, rocafella, invit, somethin, ..."
2,Forgive Me Father,"[mayb, caus, im, eatin, bastard, fiend, grub, ..."
3,Down and Out,"[ugh, killa, babi, kany, 1970, heron, flow, hu..."
4,Fly In,"[ask, young, boy, gon, second, time, around, g..."


# Generate Query (still a lot to do)